In [6]:
import numpy as np  #importing numpy
import pandas as pd #importing pandas
from sklearn.preprocessing import StandardScaler #importing standardscaler
from sklearn.model_selection import GridSearchCV, cross_validate # importing gridsearch
from sklearn.svm import SVR,SVC #importing SVr
from sklearn.model_selection import RandomizedSearchCV  #importing Randomsearch
from sklearn.model_selection import train_test_split #importing train and test
from sklearn.metrics import confusion_matrix,r2_score,f1_score #import metrics

In [7]:
df=pd.read_csv('winequality-white.csv') #reading the dataset
df1=pd.read_csv('Movie_classification.csv') #reading the dataset
df2=pd.read_csv('kc_house.csv') #reading the dataset
df3=pd.read_csv('loan_classification.csv') #reading the dataset

In [8]:
def null_replace(df): #define fun for replacing the null values
    for col in df.columns:
        if (df[col].dtypes == "object"): # checking the datatype
            df[col]=df[col].fillna(df[col].mode()[0]) #replacing the null values for catgeory column
        else:
            df[col]=df[col].fillna(df[col].mean()) # replacing the null values for numerical columns
    return df

def label_encoder(df): #define fun for label encoder 
    from sklearn.preprocessing import LabelEncoder #doing the labelencoder for category columns
    le = LabelEncoder() # giving variable for labelencoder
    for col in df.columns:  
        if (df[col].dtypes == "object"): #checking the data type of the column
            df[col] = df[col].astype(str) 
            df[col] = le.fit_transform(df[col]) #applying the label encoder
    return df #returns dataframe 

**SVR** **Regression**

In [9]:
%%time
def Svr_model(df):
    target=input(str('Enter the target variable : '))

    df=null_replace(df) #replacing the null values
    df=label_encoder(df) #replacing the label encoder

    X =df.drop(target,axis=1) # X 
    y =df[target] #y

    scaler = StandardScaler() #appling standard scaler

    X = scaler.fit_transform(X) #fitting the standard scaler 

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) #spliting train and test 

    param_grid = [{'C': [0.01, 0.1, 10, 100,1000], 'kernel': ['linear'],'epsilon':[0.2,0.5,0.6,0.8,1]},
                {'C': [0.01, 0.1, 10, 100,1000], 'gamma': [0.0001,0.001, 0.01, 1, 10, 100], 'kernel': ['rbf'],'epsilon':[0.2,0.5,0.6,0.8,1]},
                {'C': [0.01, 0.1, 10, 100,1000], 'gamma': [0.0001,0.001, 0.01, 1, 10, 100], 'kernel': ['sigmoid'],'epsilon':[0.2,0.5,0.6,0.8,1]},
                {'C': [0.01, 0.1, 10, 100,1000], 'gamma': [0.0001,0.001, 0.01, 1, 10, 100], 'kernel': ['poly'],'epsilon':[0.2,0.5,0.6,0.8,1],'degree':[2,4,5,6,7]}]
                 # parameters for svr 

    rnd_search = RandomizedSearchCV(SVR(), param_grid, n_iter =10, cv=2) #Running the random search

    rnd_search.fit(X_train,y_train) #fitting the X_train and y_train

    param = {'C': [0.01, 0.1, 10, 100,1000],  'gamma':[0.0001,0.001, 0.01, 1, 10, 100],'epsilon':[0.2,0.5,0.6,0.8,1],'degree':[2,4,5,6,7,8]} 
    #fixed parameters for SVR

    kernel=[] #Empty list of kernel
    kernel.append(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('kernel')]) 
    #accessing the kernel values through indexing from random search best parameters dictonary
    #appending the kernel from random search to empty list

    gamma=[]  #Empty list of gamma
    if 'gamma' in list(rnd_search.best_params_.keys()): #checking if the gamma key is present in the random search best parameters dictonary 

    #checking gamma value from random search best parameters in fixed parameter dict   
    #if the value of gamma is in last position telling to take last 3 position values 
    if param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])] ==param['gamma'][-1]:
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])-2])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])-1])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])])

    #if the value of gamma is in first position telling to take first 3 position values
    elif param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])] ==param['gamma'][0]:
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])+1])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])+2])

    #if the value of gamma is in between position telling to take front and back of the values of the gamma values    
    else:
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])-1])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])+1])

        #if the gamma value is not in the random search best parameters dictonary 
        #appending the default value to the list
    else:
    gamma.append('scale')


    epsilon=[] #Empty list of epsilon
    if 'epsilon' in list(rnd_search.best_params_.keys()): #checking if the epsilon key is present in the random search best parameters dictonary 


    #checking epsilon value from random search best parameters in fixed parameter dict   
    #if the value of epsilon is in last position telling to take last 3 position values 
    if param['epsilon'][param['epsilon'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('epsilon')])] == param['epsilon'][-1]:
        epsilon.append(param['epsilon'][param['epsilon'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('epsilon')])-2])
        epsilon.append(param['epsilon'][param['epsilon'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('epsilon')])-1])
        epsilon.append(param['epsilon'][param['epsilon'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('epsilon')])])


    #if the value of epsilon is in first position telling to take first 3 position values
    elif param['epsilon'][param['epsilon'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('epsilon')])] == param['epsilon'][0]:
        epsilon.append(param['epsilon'][param['epsilon'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('epsilon')])])
        epsilon.append(param['epsilon'][param['epsilon'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('epsilon')])+1])
        epsilon.append(param['epsilon'][param['epsilon'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('epsilon')])+2])

    #if the value of epsilon is in between position telling to take front and back of the values of the epsilon values 
    else:
        epsilon.append(param['epsilon'][param['epsilon'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('epsilon')])-1])
        epsilon.append(param['epsilon'][param['epsilon'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('epsilon')])])
        epsilon.append(param['epsilon'][param['epsilon'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('epsilon')])+1])


    degree=[] #Empty list of degree
    if 'degree' in list(rnd_search.best_params_.keys()): #checking if the degree key is present in the random search best parameters dictonary 

    #checking degree value from random search best parameters in fixed parameter dict   
    #if the value of degree is in last position telling to take last 3 position values
    if param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])] ==param['degree'][-1]:
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])-2])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])-1])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])])

    #if the value of degree is in first position telling to take first 3 position values
    elif param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])] == param['degree'][0]:
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])+1])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])+2])

    #if the value of degree is in between position telling to take front and back of the values of the degree values 
    else:
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])-1])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])+1])

        #if the degree value is not in the random search best parameters dictonary 
        #appending the default value to the list        
    else:
    degree.append(3)

    C=[] #Empty list of C
    if 'C' in list(rnd_search.best_params_.keys()): #checking if the C key is present in the random search best parameters dictonary


    #checking C value from random search best parameters in fixed parameter dict   
    #if the value of C is in last position telling to take last 3 position values
    if param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])] ==param['C'][-1]:
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])-2])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])-1])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])])

    #if the value of C is in first position telling to take first 3 position values
    elif param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])] == param['C'][0]:
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])+1])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])+2])

    #if the value of C is in between position telling to take front and back of the values of the C values
    else:
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])-1])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])+1])


    param_dist={'kernel':kernel,'gamma':gamma,'epsilon':epsilon,'degree':degree,'C':C} 
    #Creating the dictonary parameters values from random search 

    clf = GridSearchCV(SVR(), param_dist,cv=2, scoring='neg_mean_squared_error',n_jobs = -1) 
    #running the Gridsearch from the parameters from random search 

    clf.fit(X_train, y_train) #fitting the grid search

    svr_model = SVR(C=clf.best_params_['C'], kernel=clf.best_params_['kernel'], gamma= clf.best_params_['gamma'],degree=clf.best_params_['degree'],epsilon=clf.best_params_['epsilon'])
    #fitting the SVR 
    svr_model.fit(X_train, y_train)

    y_pred = svr_model.predict(X_test) #predicting 
    r2_score=r2_score(y_test, y_pred) #performance metric

    return r2_score


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.44 µs


In [ ]:
%%time
Svr_model(df)

**SVC** **Classification**

In [ ]:
%%time
def Svc_model(df):
    target=input(str('Enter the target variable : '))

    df=null_replace(df) #replacing the null values
    df=label_encoder(df) #replacing the label encoder

    X =df.drop(target,axis=1) # X 
    y =df[target] #y

    scaler = StandardScaler() #appling standard scaler

    X = scaler.fit_transform(X) #fitting the standard scaler 

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) #spliting train and test 


    param_grid = [{'C': [0.01, 0.1, 10, 100,1000], 'kernel': ['linear']},
                  {'C': [0.01, 0.1, 10, 100,1000], 'gamma': [0.0001,0.001, 0.01, 1, 10, 100], 'kernel': ['rbf']},
                  {'C': [0.01, 0.1, 10, 100,1000], 'gamma': [0.0001,0.001, 0.01, 1, 10, 100], 'kernel': ['sigmoid']},
                  {'C': [0.01, 0.1, 10, 100,1000], 'gamma': [0.0001,0.001, 0.01, 1, 10, 100], 'kernel': ['poly'],'degree':[2,4,5,6,7]}]
                  # parameters for svc

    rnd_search = RandomizedSearchCV(SVR(), param_grid, n_iter =10, cv=2) #Running the random search

    rnd_search.fit(X_train,y_train) #fitting the X_train and y_train

    param = {'C': [0.01, 0.1, 10, 100,1000],  'gamma':[0.0001,0.001, 0.01, 1, 10, 100],'degree':[2,4,5,6,7,8]} 
    #fixed parameters for SVC

    kernel=[] #Empty list of kernel
    kernel.append(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('kernel')]) 
    #accessing the kernel values through indexing from random search best parameters dictonary
    #appending the kernel from random search to empty list

    gamma=[]  #Empty list of gamma
    if 'gamma' in list(rnd_search.best_params_.keys()): #checking if the gamma key is present in the random search best parameters dictonary 

    #checking gamma value from random search best parameters in fixed parameter dict   
    #if the value of gamma is in last position telling to take last 3 position values 
    if param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])] ==param['gamma'][-1]:
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])-2])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])-1])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])])

    #if the value of gamma is in first position telling to take first 3 position values
    elif param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])] ==param['gamma'][0]:
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])+1])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])+2])

    #if the value of gamma is in between position telling to take front and back of the values of the gamma values    
    else:
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])-1])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])])
        gamma.append(param['gamma'][param['gamma'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('gamma')])+1])

        #if the gamma value is not in the random search best parameters dictonary 
        #appending the default value to the list
    else:
    gamma.append('scale')


    degree=[] #Empty list of degree
    if 'degree' in list(rnd_search.best_params_.keys()): #checking if the degree key is present in the random search best parameters dictonary 

    #checking degree value from random search best parameters in fixed parameter dict   
    #if the value of degree is in last position telling to take last 3 position values
    if param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])] ==param['degree'][-1]:
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])-2])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])-1])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])])

    #if the value of degree is in first position telling to take first 3 position values
    elif param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])] == param['degree'][0]:
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])+1])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])+2])

    #if the value of degree is in between position telling to take front and back of the values of the degree values 
    else:
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])-1])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])])
        degree.append(param['degree'][param['degree'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('degree')])+1])

        #if the degree value is not in the random search best parameters dictonary 
        #appending the default value to the list        
    else:
    degree.append(3)

    C=[] #Empty list of C
    if 'C' in list(rnd_search.best_params_.keys()): #checking if the C key is present in the random search best parameters dictonary


    #checking C value from random search best parameters in fixed parameter dict   
    #if the value of C is in last position telling to take last 3 position values
    if param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])] ==param['C'][-1]:
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])-2])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])-1])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])])

    #if the value of C is in first position telling to take first 3 position values
    elif param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])] == param['C'][0]:
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])+1])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])+2])

    #if the value of C is in between position telling to take front and back of the values of the C values
    else:
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])-1])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])])
        C.append(param['C'][param['C'].index(list(rnd_search.best_params_.values())[list(rnd_search.best_params_.keys()).index('C')])+1])


    param_dist={'kernel':kernel,'gamma':gamma,'degree':degree,'C':C} 
    #Creating the dictonary parameters values from random search 

    clf = GridSearchCV(SVR(), param_dist,cv=2, scoring='neg_mean_squared_error',n_jobs = -1) 
    #running the Gridsearch from the parameters from random search 

    clf.fit(X_train, y_train) #fitting the grid search

    svc_model = SVC(C=clf.best_params_['C'], kernel=clf.best_params_['kernel'], gamma= clf.best_params_['gamma'],degree=clf.best_params_['degree'])

    #fitting the SVC
    svc_model.fit(X_train, y_train)

    y_pred = svc_model.predict(X_test) #predicting 
    f1_score=f1_score(y_test, y_pred) #performance metric

    return f1_score


CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 8.58 µs


In [ ]:
Svc_model(df1)